In [1]:
from bs4 import BeautifulSoup
import os
from glob import glob

import urllib.request
import shutil
from tqdm import tqdm
from pydub import AudioSegment
import multiprocessing as mp
import pickle

## My codes
from src.utils import SoundToText
from src.utils import PrepareSound

## settings
%matplotlib inline

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
!ls

Data			      Jun8_MVPrecording.ipynb  Sphinx-VS-Google.ipynb
DownloadAudio_NPRconv.ipynb   MVP.ipynb		       src
DownloadAudio_NPR.ipynb       NLP_studies.ipynb        Text_to_Suammary.ipynb
DownloadAudio_NPRstory.ipynb  Plot		       website
Jun15_MVP.ipynb		      Prev_MVP.ipynb
Jun8_MVPProcessing.ipynb      README.md


##  Data source:
##  NPR: https://www.npr.org/podcasts/2038/news-politics

## https://www.npr.org/podcasts/452538677/note-to-self
## https://www.npr.org/podcasts/510298/ted-radio-hour

## https://www.npr.org/series/4516989/storycorps/archive

In [3]:
soup = BeautifulSoup(open("Data/NPR_conv/NoteToSelfNPR.html"), "html.parser")

In [4]:
pod_text = []
pod_audi = []

for i in soup.find_all('article', {'class' : 'item podcast-episode'}):
#for i in soup.find_all('article', {'class' : 'item has-image'}):
    #pods.append(i)
    #make sure both audi and text exist for the same episode
    for j in i.find_all('p', {'class': 'teaser'}):
        pod_text.append(j)
    for j in i.find_all('li', {'class': 'audio-tool audio-tool-download'}):
        #link = j.find_all('href')
        pod_audi.append(j)
print(pod_text[0], pod_audi[0])

<p class="teaser">
<time datetime="2018-06-07"><span class="date">June 7, 2018 • </span></time>We asked you guys to send us photos. Then we gave them to Andreas Weigend, veteran of Xerox Parc, former chief scientist at Amazon, to see what he could deduce. A lot, it turns out. A little Google image search, a little metadata, and we can find where you are. Maybe who you are. What color phone you're using to take the shot, and how many SIM cards you have. Reading photos is more than a digital parlor trick. It's the future of commerce, marketing, policing, lending, and basically everything else. ------- For the next several weeks you'll hear the "Best of" Note to Self in your podcast feed. Our favorite episodes. Manoush will be working on some other projects, but Note to Self will be back before you know it with some changes and surprises. Keep in touch with her on Twitter, Instagram, and on her website.
        </p> <li class="audio-tool audio-tool-download"><a data-metrics='{"category":"

In [5]:
pod_text = []
pod_audi = []

for i in soup.find_all('article', {'class' : 'item podcast-episode'}):
#for i in soup.find_all('article', {'class' : 'item has-image'}):
    #pods.append(i)
    #make sure both audi and text exist for the same episode
    text = i.find_all('p', {'class': 'teaser'})
    audi = i.find_all('li', {'class': 'audio-tool audio-tool-download'})
    if len(text) != 1 or len(audi) != 1:
        ## no download links...
        pass
        #print (len(text), len(audi))
    else:
        pod_text.append(text[0])
        pod_audi.append(audi[0].find_all('a', href=True)[0])
print(pod_text[0], pod_audi[0])

<p class="teaser">
<time datetime="2018-06-07"><span class="date">June 7, 2018 • </span></time>We asked you guys to send us photos. Then we gave them to Andreas Weigend, veteran of Xerox Parc, former chief scientist at Amazon, to see what he could deduce. A lot, it turns out. A little Google image search, a little metadata, and we can find where you are. Maybe who you are. What color phone you're using to take the shot, and how many SIM cards you have. Reading photos is more than a digital parlor trick. It's the future of commerce, marketing, policing, lending, and basically everything else. ------- For the next several weeks you'll hear the "Best of" Note to Self in your podcast feed. Our favorite episodes. Manoush will be working on some other projects, but Note to Self will be back before you know it with some changes and surprises. Keep in touch with her on Twitter, Instagram, and on her website.
        </p> <a data-metrics='{"category":"audio actions","action":"download audio","l

In [6]:
print(len(pod_text), len(pod_audi))
N = len(pod_text)

96 96


In [7]:
t = pod_text[0]

In [8]:
## Get teaser
str(t).split("</time>")[1].split("\n")[0]

'We asked you guys to send us photos. Then we gave them to Andreas Weigend, veteran of Xerox Parc, former chief scientist at Amazon, to see what he could deduce. A lot, it turns out. A little Google image search, a little metadata, and we can find where you are. Maybe who you are. What color phone you\'re using to take the shot, and how many SIM cards you have. Reading photos is more than a digital parlor trick. It\'s the future of commerce, marketing, policing, lending, and basically everything else. ------- For the next several weeks you\'ll hear the "Best of" Note to Self in your podcast feed. Our favorite episodes. Manoush will be working on some other projects, but Note to Self will be back before you know it with some changes and surprises. Keep in touch with her on Twitter, Instagram, and on her website.'

In [9]:
a = pod_audi[0]

In [10]:
a

<a data-metrics='{"category":"audio actions","action":"download audio","label":"617787704","noninteraction":true}' href="https://www.podtrac.com/pts/redirect.mp3/audio.wnyc.org/notetoself/notetoself060718_cms860748_pod.mp3?siteplayer=true&amp;dl=1"><b class="icn-download"></b><b class="audio-tool-label">Download</b></a>

In [11]:
## Get download link
a.get("href")

'https://www.podtrac.com/pts/redirect.mp3/audio.wnyc.org/notetoself/notetoself060718_cms860748_pod.mp3?siteplayer=true&dl=1'

In [12]:
!ls Data/NPR_conv

0.mp3	20.txt	32.mp3	43.txt	55.mp3	66.txt	78.mp3	89.txt
0.txt	21.mp3	32.txt	44.mp3	55.txt	67.mp3	78.txt	8.mp3
10.mp3	21.txt	33.mp3	44.txt	56.mp3	67.txt	79.mp3	8.txt
10.txt	22.mp3	33.txt	45.mp3	56.txt	68.mp3	79.txt	90.mp3
11.mp3	22.txt	34.mp3	45.txt	57.mp3	68.txt	7.mp3	90.txt
11.txt	23.mp3	34.txt	46.mp3	57.txt	69.mp3	7.txt	91.mp3
12.mp3	23.txt	35.mp3	46.txt	58.mp3	69.txt	80.mp3	91.txt
12.txt	24.mp3	35.txt	47.mp3	58.txt	6.mp3	80.txt	92.mp3
13.mp3	24.txt	36.mp3	47.txt	59.mp3	6.txt	81.mp3	92.txt
13.txt	25.mp3	36.txt	48.mp3	59.txt	70.mp3	81.txt	93.mp3
14.mp3	25.txt	37.mp3	48.txt	5.mp3	70.txt	82.mp3	93.txt
14.txt	26.mp3	37.txt	49.mp3	5.txt	71.mp3	82.txt	94.mp3
15.mp3	26.txt	38.mp3	49.txt	60.mp3	71.txt	83.mp3	94.txt
15.txt	27.mp3	38.txt	4.mp3	60.txt	72.mp3	83.txt	95.mp3
16.mp3	27.txt	39.mp3	4.txt	61.mp3	72.txt	84.mp3	95.txt
16.txt	28.mp3	39.txt	50.mp3	61.txt	73.mp3	84.txt	9.mp3
17.mp3	28.txt	3.mp3	50.txt	62.mp3	73.txt	85.mp3	9.txt
17.txt	29.mp3	3.txt	51.mp3	62.txt	74.mp3	85.txt	NoteToSelfNPR.h

In [15]:
path = "Data/NPR_conv/"
for i in tqdm(range(0, len(pod_text))):
    t = pod_text[i]
    ## Get time string
    i_time = t.find("time").get("datetime").replace("-", "")
    ## Get teaser
    i_teaser = str(t).split("</time>")[1].split("\n")[0]
    a = pod_audi[i]
    ## Get download link
    i_url = a.get("href")
    
    with open(path + str(i) + ".txt", "w") as text_file:
        text_file.write(i_teaser)
        text_file.write("\n")
        text_file.write(i_time)
    
    ## download file
    output_file = path + str(i) + ".mp3"
    with urllib.request.urlopen(i_url) as response, open(output_file, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)

#     sound = AudioSegment.from_mp3(output_file)
    
#     ## split into 20 second pieces
#     for j, chunk in enumerate(sound[::20 * 1000]):
#       with open(path + str(i) + "_" + str(j) + ".wav", "wb") as f:
#         chunk.export(f, format="wav")

100%|██████████| 96/96 [43:47<00:00, 27.37s/it] 


In [ ]:
path = "Data/NPR_conv/"
bigdic = {}
for i in tqdm(range(0, N)):
    bigdic.update(PrepareSound(path + str(i) + ".mp3", silencesplit=True))
    file_path = path ## total 
    inputtasks = glob(file_path + str(i) + "_*.wav")
    #print(" Running %s jobs on %s cores" % (len(inputtasks), mp.cpu_count()-2))
    npool = min(len(inputtasks), mp.cpu_count())
    pool  = mp.Pool(npool)

    fulltextdic = {}
    ## parallel
    for result in pool.map(SoundToText, inputtasks):
            fulltextdic.update(result)
#     ## standard
#     for j in inputtasks:
#         print(j)
#         result = SoundToText(j) #dictionary of values, plots
#         fulltextdic.update(result)
    
    for k in inputtasks:
        os.remove(k)
        
    fulltext = ""
    for l in range(len(fulltextdic.keys())):
        fulltext += fulltextdic[file_path + str(i) + "_" + str(l) + ".wav"] + " ; "
    #print(fulltext)
    with open(file_path + str(i) + "_trans.txt", "w") as text_file:
        text_file.write(fulltext)
    with open(file_path + 'info_' + str(i) + '.pkl', 'wb') as f:
        pickle.dump(fulltextdic, f)
    pool.close()

In [16]:
bigdic = {}
for i in tqdm(range(N)):
    bigdic.update(PrepareSound(path + str(i) + ".mp3", silencesplit=True))

100%|██████████| 426/426 [07:19<00:00,  1.03s/it]


In [18]:
with open(path + 'info.pkl', 'wb') as f:
    pickle.dump(bigdic, f)

In [19]:
with open(path + 'info.pkl', 'rb') as f:
    mydic = pickle.load(f)

In [22]:
mydic[path + "0.wav"]

{'dBFS': -20.927292716731948,
 'max_dBFS': -1.1586272649143894,
 'duration': 148.89795918367346,
 'rms': 2945,
 'max': 28676}

## clean the tail files
#### find *.wav -type f -size -500k -delete

In [23]:
#start_time = time.time()
## 228 is missing
for i in tqdm(range(0, N)): #len(pod_text) 
    file_path = path ## total 
    inputtasks = glob(file_path + str(i) + "_*.wav")
    #print(" Running %s jobs on %s cores" % (len(inputtasks), mp.cpu_count()-2))
    npool = min(len(inputtasks), mp.cpu_count()-1)
    pool  = mp.Pool(npool)

    fulltextdic = {}
    ## parallel
    for result in pool.map(SoundToText, inputtasks):
            fulltextdic.update(result)
#     ## standard
#     for j in inputtasks:
#         print(j)
#         result = SoundToText(j) #dictionary of values, plots
#         fulltextdic.update(result)
    
    fulltext = ""
    for l in range(len(fulltextdic.keys())):
        fulltext += fulltextdic[file_path + str(i) + "_" + str(l) + ".wav"] + " ; "
    #print(fulltext)
    with open(file_path + str(i) + "_trans.txt", "w") as text_file:
        text_file.write(fulltext)
    with open(file_path + 'info_' + str(i) + '.pkl', 'wb') as f:
        pickle.dump(fulltextdic, f)
    pool.close()
#print("--- %s seconds ---" % (time.time() - start_time))

  1%|          | 3/426 [01:04<2:30:26, 21.34s/it]

cannot recognize  Data/NPR_story/3_55.wav


  3%|▎         | 14/426 [04:53<2:24:04, 20.98s/it]

cannot recognize  Data/NPR_story/14_53.wav


 21%|██        | 88/426 [30:12<1:56:01, 20.60s/it]

...

 95%|█████████▌| 406/426 [2:26:31<07:13, 21.65s/it]

cannot recognize  Data/NPR_story/406_54.wav


 99%|█████████▉| 423/426 [2:32:43<01:04, 21.66s/it]

cannot recognize  Data/NPR_story/423_50.wav


100%|██████████| 426/426 [2:33:52<00:00, 21.67s/it]


## Finished!